In [1]:
import os
import tqdm
import os.path as osp
from tcre.env import *
import itertools
%run ../../../src/tcre/logging.py

In [2]:
output_dir = osp.join(RESULTS_DATA_DIR, 'label-modeling-v3', 'run1')
args = []
for p in itertools.product(
    ['mult_01', 'sngl_01', 'doub_01'], 
    [True, False],
    [True, False],
    ['w2v_frozen', 'w2v_trained', 'denovo'],
    ['S', 'L'],
    [0, .005]
):
    secondary = 'use-secondary' if p[1] else 'no-secondary'
    swaps = 'use-swaps' if p[2] else 'no-swaps'
    args.append(f"--marker-list={p[0]} --{secondary} --{swaps} --wrd-embedding-type={p[3]} --model-size={p[4]} --weight-decay={p[5]}")
args[:5]

['--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=S --weight-decay=0',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=S --weight-decay=0.005',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0.005',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_trained --model-size=S --weight-decay=0']

In [2]:
output_dir = osp.join(RESULTS_DATA_DIR, 'label-modeling-v3', 'run2')
args = []
for p in itertools.product(
    ['mult_01', 'doub_01'], 
    [False],
    [True],
    ['w2v_frozen', 'w2v_trained'],
    ['L', 'XL'],
    [0, .001, .005],
    [True, False],
    [0.0, 0.5]
):
    secondary = 'use-secondary' if p[1] else 'no-secondary'
    swaps = 'use-swaps' if p[2] else 'no-swaps'
    positions = 'use-positions' if p[6] else 'no-positions'
    args.append(f"--marker-list={p[0]} --{secondary} --{swaps} --wrd-embedding-type={p[3]} --model-size={p[4]} --weight-decay={p[5]} --{positions} --dropout={p[7]}")
args[:5]

['--marker-list=mult_01 --no-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0 --use-positions --dropout=0.0',
 '--marker-list=mult_01 --no-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0 --use-positions --dropout=0.5',
 '--marker-list=mult_01 --no-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0 --no-positions --dropout=0.0',
 '--marker-list=mult_01 --no-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0 --no-positions --dropout=0.5',
 '--marker-list=mult_01 --no-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0.001 --use-positions --dropout=0.0']

In [3]:
cmd_format = """python /lab/repos/t-cell-relation-extraction/src/tcre/exec/v1/train.py \
--relation-class={relation_class} --device={device} --output-dir={output_dir} \
{args} > {log_file} 2>&1
"""

rc_types = ['inducing_transcription_factor', 'secreted_cytokine', 'inducing_cytokine']

def get_cmd(rc, args, i):
    out_dir = osp.join(output_dir, f'config_{rc}_{i:04d}')
    log_file = osp.join(out_dir, f'log.txt')
    if not osp.exists(out_dir):
        os.makedirs(out_dir)
    cmd = cmd_format.format(relation_class=rc, device='"cuda:1"', output_dir=out_dir, log_file=log_file, args=args)
    return cmd
    
    
cmds = [get_cmd(rc, arg, i) for rc in rc_types for i, arg in enumerate(args)]

print('Sample commands:')
for cmd in cmds[:10]:
    print(cmd)

for cmd in tqdm.tqdm(cmds):
    rc = os.system(cmd)
    if rc != 0:
        raise ValueError(f'Return code {rc} (!=0) for command: {cmd}')

  0%|          | 0/288 [00:00<?, ?it/s]

Sample commands:
python /lab/repos/t-cell-relation-extraction/src/tcre/exec/v1/train.py --relation-class=inducing_transcription_factor --device="cuda:1" --output-dir=/lab/data/results/label-modeling-v3/run2/config_inducing_transcription_factor_0000 --marker-list=mult_01 --no-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0 --use-positions --dropout=0.0 > /lab/data/results/label-modeling-v3/run2/config_inducing_transcription_factor_0000/log.txt 2>&1

python /lab/repos/t-cell-relation-extraction/src/tcre/exec/v1/train.py --relation-class=inducing_transcription_factor --device="cuda:1" --output-dir=/lab/data/results/label-modeling-v3/run2/config_inducing_transcription_factor_0001 --marker-list=mult_01 --no-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0 --use-positions --dropout=0.5 > /lab/data/results/label-modeling-v3/run2/config_inducing_transcription_factor_0001/log.txt 2>&1

python /lab/repos/t-cell-relation-e

100%|██████████| 288/288 [1:54:22<00:00, 23.62s/it]


In [ ]:
!python /lab/repos/t-cell-relation-extraction/src/tcre/exec/v1/train.py \
--relation-class=inducing_transcription_factor \
--marker-list=mult_01 \
--use-secondary \
--use-swaps \
--wrd-embedding-type=w2v_trained \
--model-size=L \
--weight-decay=.005 \
--device='cuda:1' \
--output-dir='/tmp/test'

In [97]:
!ls /tmp/test

config.json  history.json
